In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cricket-shots-ipl-2023/pull/32_pbks_dc_pull_9.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/31_rcb_lsg_pull_1.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/35_gt_kkr_pull_10.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/25_kkr_srh_pull_13.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/15_dc_lsg_pull_5.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/18_mi_rcb_pull_1.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/35_gt_kkr_pull_8.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/16_pbks_gt_pull_11.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/8_pbks_kkr_pull_2.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/20_rr_lsg_pull_5.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/5_rr_srh_pull_4.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/39_rr_rcb_pull_6.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/2_mi_dc_pull_7.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/33_mi_csk_pull_1.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/16_pbks_gt_pull_6.mp4
/kaggle/input/cricket-shots-ipl-2023/pull/22_csk_

In [2]:
import os
import shutil

folder_path = "/kaggle/working"

# Iterate and remove each item in the directory
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")

In [3]:
!pip install opencv-python

In [4]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_interval=1, max_images=500):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0
    success, frame = cap.read()

    os.makedirs(output_folder, exist_ok=True)
    
    while success and saved_count < max_images:
        if frame_count % frame_interval == 0:
            frame_path = os.path.join(output_folder, f"{os.path.basename(video_path).split('.')[0]}_frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1
        
        success, frame = cap.read()
        frame_count += 1
    
    cap.release()
    return saved_count

shots = ['cut', 'drive', 'flick', 'misc', 'pull', 'slog', 'sweep']
total_images = 0
frame_interval = 1  
max_images_per_shot = 500  # Set a maximum of 500 images per shot

for shot in shots:
    video_folder = f'/kaggle/input/cricket-shots-ipl-2023/{shot}'
    output_folder = f'/kaggle/working/images/{shot}_shots/'
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    else:
        for f in os.listdir(output_folder):
            os.remove(os.path.join(output_folder, f))
    
    images_generated_for_shot = 0
    for video_file in os.listdir(video_folder):
        video_path = os.path.join(video_folder, video_file)
        images_from_video = extract_frames(video_path, output_folder, frame_interval, max_images=max_images_per_shot - images_generated_for_shot)
        
        images_generated_for_shot += images_from_video
        total_images += images_from_video
        
        # Stop processing if we've reached the limit of 500 images for this shot
        if images_generated_for_shot >= max_images_per_shot:
            break

print(f"Total images generated: {total_images}")

Total images generated: 3500


In [5]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

for shot in shots:
    frame_dir = f'/kaggle/working/images/{shot}_shots/'
    label_dir = f'/kaggle/working/labels/{shot}_shots/'
    
    os.makedirs(label_dir, exist_ok=True)
    
    for frame_name in os.listdir(frame_dir):
        img_path = os.path.join(frame_dir, frame_name)
        
        results = model(img_path)
        
        for detection in results.xywh[0]:
            x_center, y_center, width, height, confidence, class_id = detection.tolist()
            if class_id == 0:
                annotation_path = os.path.join(label_dir, f"{frame_name.split('.')[0]}.txt")
                with open(annotation_path, 'w') as f:
                    f.write(f"{shots.index(shot)} {x_center} {y_center} {width} {height}\n")

/opt/conda/lib/python3.10/site-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 16.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-10-29 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 127MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarni

In [6]:
import shutil
import random

split_ratio = 0.8

for shot in shots:
    source_dir = f'/kaggle/working/images/{shot}_shots/'
    train_dir = f'/kaggle/working/images/{shot}_shots/train/'
    val_dir = f'/kaggle/working/images/{shot}_shots/val/'

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    all_images = [f for f in os.listdir(source_dir) if f.endswith(('.jpg', '.png'))]
    random.shuffle(all_images)
    
    split_index = int(len(all_images) * split_ratio)
    train_images = all_images[:split_index]
    val_images = all_images[split_index:]

    for img in train_images:
        shutil.move(os.path.join(source_dir, img), os.path.join(train_dir, img))
    
    for img in val_images:
        shutil.move(os.path.join(source_dir, img), os.path.join(val_dir, img))
    
    print(f"{shot.capitalize()} - Moved {len(train_images)} images to training set and {len(val_images)} images to validation set.")

Cut - Moved 400 images to training set and 100 images to validation set.
Drive - Moved 400 images to training set and 100 images to validation set.
Flick - Moved 400 images to training set and 100 images to validation set.
Misc - Moved 400 images to training set and 100 images to validation set.
Pull - Moved 400 images to training set and 100 images to validation set.
Slog - Moved 400 images to training set and 100 images to validation set.
Sweep - Moved 400 images to training set and 100 images to validation set.


In [7]:
for shot in shots:
    source_labels_dir = f'/kaggle/working/labels/{shot}_shots/'
    train_labels_dir = f'/kaggle/working/labels/{shot}_shots/train/'
    val_labels_dir = f'/kaggle/working/labels/{shot}_shots/val/'

    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    train_images = [f for f in os.listdir(f'/kaggle/working/images/{shot}_shots/train/') if f.endswith(('.jpg', '.png'))]

    for img in train_images:
        label_file = os.path.splitext(img)[0] + '.txt'
        if os.path.isfile(os.path.join(source_labels_dir, label_file)):
            shutil.move(os.path.join(source_labels_dir, label_file), os.path.join(train_labels_dir, label_file))
        else:
            print(f"Label file not found for: {img}")

    val_images = [f for f in os.listdir(f'/kaggle/working/images/{shot}_shots/val/') if f.endswith(('.jpg', '.png'))]

    for img in val_images:
        label_file = os.path.splitext(img)[0] + '.txt'
        if os.path.isfile(os.path.join(source_labels_dir, label_file)):
            shutil.move(os.path.join(source_labels_dir, label_file), os.path.join(val_labels_dir, label_file))
        else:
            print(f"Label file not found for: {img}")

    print(f"{shot.capitalize()} - Labels have been moved to the respective train and validation directories.")

Cut - Labels have been moved to the respective train and validation directories.
Drive - Labels have been moved to the respective train and validation directories.
Flick - Labels have been moved to the respective train and validation directories.
Misc - Labels have been moved to the respective train and validation directories.
Pull - Labels have been moved to the respective train and validation directories.
Slog - Labels have been moved to the respective train and validation directories.
Sweep - Labels have been moved to the respective train and validation directories.


In [8]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 25.92 MiB/s, done.
Resolving deltas: 100% (11695/11695), done.


In [9]:
!pip install -r yolov5/requirements.txt

In [10]:
data_yaml = f'''
train: /kaggle/working/images
val: /kaggle/working/images

nc: 7
names:
  0: cut
  1: drive
  2: flick
  3: misc
  4: pull
  5: slog
  6: sweep
'''

In [11]:
with open('/kaggle/working/cricket_shots.yaml', 'w') as f:
    f.write(data_yaml)

In [12]:
!rm -rf /kaggle/working/images/train.cache
!rm -rf /kaggle/working/images/val.cache

In [13]:
import os
import cv2

def normalize_bounding_boxes(image_path, label_path):
    # Load image to get dimensions
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    with open(label_path, 'r') as file:
        lines = file.readlines()

    normalized_lines = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        
        # Convert absolute coordinates to normalized values (0 to 1)
        x_center /= w
        y_center /= h
        width /= w
        height /= h

        if 0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1:
            normalized_lines.append(f"{int(class_id)} {x_center} {y_center} {width} {height}\n")
        else:
            print(f"Warning: Skipped out-of-bounds bounding box in {label_path}")

    with open(label_path, 'w') as file:
        file.writelines(normalized_lines)

def process_all_labels(shots):
    for shot in shots:
        image_dir_train = f'/kaggle/working/images/{shot}_shots/train'
        label_dir_train = f'/kaggle/working/labels/{shot}_shots/train'
        image_dir_val = f'/kaggle/working/images/{shot}_shots/val'
        label_dir_val = f'/kaggle/working/labels/{shot}_shots/val'
        
        # Process training labels
        for img_file in os.listdir(image_dir_train):
            if img_file.endswith('.jpg'):
                img_path = os.path.join(image_dir_train, img_file)
                label_path = os.path.join(label_dir_train, img_file.replace('.jpg', '.txt'))
                
                if os.path.exists(label_path):
                    normalize_bounding_boxes(img_path, label_path)

        # Process validation labels
        for img_file in os.listdir(image_dir_val):
            if img_file.endswith('.jpg'):
                img_path = os.path.join(image_dir_val, img_file)
                label_path = os.path.join(label_dir_val, img_file.replace('.jpg', '.txt'))
                
                if os.path.exists(label_path):
                    normalize_bounding_boxes(img_path, label_path)

# List of all shot categories
shots = ['cut', 'drive', 'flick', 'misc', 'pull', 'slog', 'sweep']

# Process all labels for the specified shots
process_all_labels(shots)

In [14]:
!python yolov5/train.py --img 640 --batch 16 --epochs 20 --data /kaggle/working/cricket_shots.yaml --weights yolov5/yolov5s.pt --cache --name cricket_shots_training

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5/yolov5s.pt, cfg=, data=/kaggle/working/cricket_shots.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=cricket_shots_training, exist_ok=False, quad=False, c

In [15]:
import torch
import os

# Load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/cricket_shots_training/weights/best.pt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define shots and initialize evaluation metrics
shots = ['cut', 'drive', 'flick', 'misc', 'pull', 'slog', 'sweep']
all_detections = []
all_ground_truths = []

# Loop through each shot type
for shot in shots:
    # Define paths for validation images and labels for each shot
    val_images_dir = f'/kaggle/working/images/{shot}_shots/val/'
    val_labels_dir = f'/kaggle/working/labels/{shot}_shots/val/'

    # Iterate over each image in the validation folder for the shot type
    for image_file in os.listdir(val_images_dir):
        img_path = os.path.join(val_images_dir, image_file)
        results = model(img_path)

        # Load corresponding ground truth labels
        label_file = os.path.join(val_labels_dir, os.path.splitext(image_file)[0] + '.txt')
        if not os.path.exists(label_file):
            continue

        ground_truth_boxes = []
        with open(label_file, 'r') as f:
            for line in f.readlines():
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                ground_truth_boxes.append([class_id, x_center, y_center, width, height])

        # Convert detections and ground truth to tensor
        detections = results.xyxy[0]  # xyxy format (x1, y1, x2, y2, confidence, class)
        detections = detections.cpu().numpy() if torch.cuda.is_available() else detections.numpy()

        # Append detections and ground truths for evaluation
        all_detections.append(detections)
        all_ground_truths.append(ground_truth_boxes)

# Now, evaluate metrics manually (e.g., precision, recall, and mAP)
def calculate_metrics(detections, ground_truths, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    # Loop through each detection and ground truth pair
    for dets, gts in zip(detections, ground_truths):
        for det in dets:
            x1, y1, x2, y2, conf, class_id = det
            best_iou = 0
            best_gt = None

            # Check against ground truths
            for gt in gts:
                gt_class_id, gt_xc, gt_yc, gt_width, gt_height = gt
                # Convert ground truth from center format to corner format
                gt_x1 = gt_xc - gt_width / 2
                gt_y1 = gt_yc - gt_height / 2
                gt_x2 = gt_xc + gt_width / 2
                gt_y2 = gt_yc + gt_height / 2

                # Calculate IoU
                inter_x1 = max(x1, gt_x1)
                inter_y1 = max(y1, gt_y1)
                inter_x2 = min(x2, gt_x2)
                inter_y2 = min(y2, gt_y2)
                intersection = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
                union = (x2 - x1) * (y2 - y1) + (gt_x2 - gt_x1) * (gt_y2 - gt_y1) - intersection
                iou = intersection / union if union > 0 else 0

                # Check if it's the best IoU
                if iou > best_iou:
                    best_iou = iou
                    best_gt = gt

            # Determine if the detection is a true positive or false positive
            if best_iou >= iou_threshold and best_gt is not None:
                true_positives += 1
            else:
                false_positives += 1

        # Count false negatives
        false_negatives += len(gts) - true_positives

    # Calculate precision and recall
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0

    return precision, recall

# Calculate metrics for the whole dataset
precision, recall = calculate_metrics(all_detections, all_ground_truths)

# Print evaluation results
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-29 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:89

Precision: 0.000
Recall: 0.000
